In [230]:
import numpy as np
import pandas as pd

In [231]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [232]:
movies = movies.merge(credits,on='title')

In [233]:
# genres
# id
# keywords
# title
# overview
# cast
# crew

movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [234]:
# only three columns - movie_id,tags, title
# tgas = merge of overview, genres, cast, crew, keywords

In [235]:
movies.dropna(inplace=True)

In [236]:
# helper function
import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [237]:
movies['genres'] = movies['genres'].apply(convert)

In [238]:
movies['keywords'] = movies['keywords'].apply(convert)

In [239]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
            counter+=1
        else:
            break;
  
    return L

In [240]:
movies['cast'] = movies['cast'].apply(convert3)

In [241]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [242]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [243]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [244]:
movies['genres'] = movies['genres'].apply( lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply( lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply( lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply( lambda x:[i.replace(" ","") for i in x])

In [245]:
movies['tags'] = movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [246]:
new_df = movies[['movie_id','title','tags']]

In [247]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\Rahil Kumar\AppData\Local\Temp\ipykernel_26608\3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [248]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\Rahil Kumar\AppData\Local\Temp\ipykernel_26608\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [249]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [250]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [251]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [252]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
 
    return " ".join(y)



In [253]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\Rahil Kumar\AppData\Local\Temp\ipykernel_26608\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [254]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(vectors)

array([[1.        , 0.08964215, 0.05976143, ..., 0.02519763, 0.02817181,
        0.        ],
       [0.08964215, 1.        , 0.0625    , ..., 0.02635231, 0.        ,
        0.        ],
       [0.05976143, 0.0625    , 1.        , ..., 0.02635231, 0.        ,
        0.        ],
       ...,
       [0.02519763, 0.02635231, 0.02635231, ..., 1.        , 0.0745356 ,
        0.04836508],
       [0.02817181, 0.        , 0.        , ..., 0.0745356 , 1.        ,
        0.05407381],
       [0.        , 0.        , 0.        , ..., 0.04836508, 0.05407381,
        1.        ]])

In [255]:
similarity = cosine_similarity(vectors)

In [256]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key = lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
    

In [257]:
recommend('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin
Batman


In [258]:
import pickle

In [259]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [260]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [261]:
pickle.dump(similarity,open('similarity.pkl','wb'))